In [4]:
#import pyvisa
#from pyvisa.constants import StopBits, Parity
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import datetime as dt
import json
import pickle

#from pyvisa.constants import StopBits, Parity

In [ ]:
rm = pyvisa.ResourceManager()

In [ ]:
rm.list_resources()

In [ ]:
class TemperatureControlller:
    def __init__(self, type_tc: str = "room", address: str = ""):
        self.type_tc = type_tc
        if type_tc == "LSCI":
            self.device = rm.open_resource(
                address,
                baud_rate=9600,
                data_bits=7,
                parity=Parity.odd,
                stop_bits=StopBits.one,
                read_termination="\r\n",
            )
            return self.device.query("*IDN?")
        elif type_tc == "Lakeshore":
            pass
        else:
            self.device = None

    def __call__(self, channel: str = "A"):
        if self.type_tc == "room":
            return str(22 + 273.15)
        if channel == "AB":
            return (
                str(float(self.device.query("KRDG? A").strip()))
                + "o"
                + str(float(self.device.query("KRDG? B").strip()))
            )
        else:
            return str(float(self.device.query("KRDG? " + channel).strip()))

    def idn(self):
        if self.type_tc != "room":
            return self.device.query("*IDN?")
        else:
            return "RoomTemperatureController"

In [ ]:
oscil = rm.open_resource("TCPIP0::192.168.1.45::INSTR")
gen = rm.open_resource("USB0::0x0699::0x0356::C020295::INSTR")
temp = TemperatureControlller(type_tc="room")

In [ ]:
print("Oscilloscope:", oscil.query("*IDN?"))
print("Generator:", gen.query("*IDN?"))
print("Temperature controller:", temp.idn())

In [ ]:
path = "" # path to save data, str

expnum = "" # number of the experiment, str
sample = "" # sample id, str
width = "" # probe width, str
lead = "" # probe lead, str
trail = "" # probe trail, str
timeimp = "" # distance between laser pulses, 2sec, etc., str
amplimp = "" # probe amplitude
laserpower = "" # percent "o" mW on ONE OUTPUT times 100 -- set on PHAROS, str
sysdelay="" # system delay, 40 \mu s +- N ns, e. g. 40003
channel_count = 3 # V_1, V_2, photodiode
trial_count = 1 # how many trials to record one after another, int
trial_writes = 20 # point to average
power_divider = "" # a coeff for PHAROS power to power on the sample conversion, str
oscil.timeout = 10000 # 10 sec, has to larger than timeimp

In [ ]:
times = []

times = [int(i) for i in times]

for i in range(len(times)):
    times[i] += int(sysdelay)

print(times)

In [ ]:
oscil.write(":TRIGger:SWEep SINGle")
oscil.write(":TIMebase:SCALe 500E-9")
oscil.write(":TIMebase:POSition 0")
oscil.write(":ACQuire:SRATe:ANALog MAX")
for i in range(trial_count):
    folder = path + str(i) + "/"
    try:
        os.mkdir(path)
    except:
        pass
    try:
        os.mkdir(folder)
    except:
        pass
    for i in tqdm(times):
        raw_data = {'ch1': '0', 'ch2': '0','ch3':'0'}
        for j in range(1,channel_count+1):
          temp_zabiv = []
          for k in range(trial_writes):
            temp_zabiv.append([])
          raw_data['ch'+str(j)] = temp_zabiv

        measurment_per_num = {'ch1': '0', 'ch2':'0','ch3':'0'}
        for k in range(trial_writes):
            num = str(k)
            #measurment_per_num = {'ch1': [a_1,a_2,...],'ch2': ...}
            current_heater = temp(channel="AB")
            timestr = str(i) + "ns"
            gen.write(":BURSt:TDELay " + timestr)
            real_delay = gen.query(":BURSt:TDELay?").strip()

            # Blocking call SINGLE
            tmp = oscil.query(":DIGitize;*OPC?")

            # Preamble save
            preamble = oscil.query(":WAVeform:PREamble?")

            with open(
                folder
                + f"{expnum}_preamble_{real_delay}_generatordelay_{sysdelay}_powdiv_{power_divider}.txt",
                "w",
            ) as f:
                for i in len(preamble.split(','))


                m = preamble.split(',')
                preamble_data = "<format>, <type>, <points>, <count> , <X increment>, <X origin>, <X reference>, <Y increment>, <Y origin>, <Y reference>, <coupling>, <X display range>, <X display origin>, <Y display range>, <Y display origin>, <date>, <time>, <frame model #>, <acquisition mode>, <completion>, <X units>, <Y units>, <max bandwidth limit>, <min bandwidth limit>"
                preamble_data_m = preamble_data.split(',')


                preambule_dict = {'<format>':'0'}
                for i in range(len(m)):
                    preambule_dict[preamble_data_m[i]] = m[i]
                for key, value in preambule_dict.items():
                  f.write(f'{key}: {value}\n')




        # Work N channels
            for channel in range(channel_count):
                ch = str(channel + 1)
                oscil.write(":WAVeform:SOURce CHAN" + ch)
                response = oscil.query(":WAVeform:SOURce?")
                if response.strip() == ("CHAN" + ch):
                    a = oscil.query(":WAVeform:DATA?")
                    tmp_data = raw_data.get('ch'+ch)
                    tmp_data[num] = a
                    raw_data['ch'+ch] = tmp_data
                else:
                    print(response.strip(), "I died")
                    break

        attribs = {'expnum':expnum, 'sample':sample,'real_delay':real_delay,'current_heater':current_heater,'width':width, 'lead':lead, 'trail':trail,'timeimp':timeimp, 'laserpower':laserpower, 'amplimp':amplimp, 'raw_data':raw_data}
        with open(str(channel), "w", encoding="utf-8") as file:
          json.dump(attribs, file)
